# VeriRegime - 4小时波动率预测CNN训练

本notebook使用模块化组件训练4小时波动率预测模型

**任务**: 预测未来4小时BTC波动率（低/高）  
**阈值**: 0.05% (平衡分布，训练稳定)  
**标签分布**: 低波动50% vs 高波动50%  
**目标**: 准确率 ≥ 65%，理想 ≥ 70%


## 1. 导入依赖


In [ ]:
import sys
sys.path.append('..')  # 添加项目根目录

import torch
import platform

# 导入训练模块
from train import CNNVolatility, create_dataloaders, Trainer

print(f"PyTorch版本: {torch.__version__}")
print(f"系统: {platform.system()}")


## 2. 配置参数


In [ ]:
# 数据配置 - 4小时波动率预测
FEATURE_COLS = ['ema_5', 'ema_10', 'ema_20', 'rsi', 'macd', 'volume_ma_5', 'volume_ma_10']
SEQ_LENGTH = 240  # 4小时 = 240分钟
BATCH_SIZE = 512

# 训练配置
EPOCHS = 50
LR = 1e-3
EARLY_STOP_PATIENCE = 10

# 路径配置
TRAIN_CSV = '../data/train.csv'
VAL_CSV = '../data/val.csv'
TEST_CSV = '../data/test.csv'

# 设备配置
if platform.system() == 'Darwin' and torch.backends.mps.is_available():
    DEVICE = torch.device('mps')
    print("🍎 使用MPS (Apple Silicon)")
elif torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print("🚀 使用CUDA")
else:
    DEVICE = torch.device('cpu')
    print("💻 使用CPU")

print(f"\n配置:")
print(f"  预测窗口: {SEQ_LENGTH}分钟 (4小时)")
print(f"  波动率阈值: 0.05%")
print(f"  批次大小: {BATCH_SIZE}")
print(f"  标签分布: ~50% 低波动, ~50% 高波动")


## 3. 加载数据


In [ ]:
print("加载数据...")

train_loader, val_loader, test_loader, class_weights, stats = create_dataloaders(
    train_csv=TRAIN_CSV,
    val_csv=VAL_CSV,
    test_csv=TEST_CSV,
    feature_cols=FEATURE_COLS,
    seq_length=SEQ_LENGTH,
    batch_size=BATCH_SIZE
)

print(f"\n数据统计:")
print(f"  训练集: {stats['train_size']:,} 样本")
print(f"  验证集: {stats['val_size']:,} 样本") 
print(f"  测试集: {stats['test_size']:,} 样本")
print(f"  类别权重: {[f'{w:.3f}' for w in class_weights]}")


## 4. 创建模型


In [ ]:
model = CNNVolatility(input_channels=len(FEATURE_COLS), seq_length=SEQ_LENGTH)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"模型参数量: {total_params:,}")


## 5. 训练模型


In [ ]:
config = {'lr': LR, 'weight_decay': 1e-4}

trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    device=DEVICE,
    class_weights=class_weights,
    config=config
)

print("\n开始训练...")
best_f1 = trainer.train(epochs=EPOCHS, early_stop_patience=EARLY_STOP_PATIENCE)


## 6. 测试模型


In [ ]:
print("\n测试最佳模型...")

test_acc, test_f1 = trainer.test()

if test_acc >= 0.60:
    print(f"\n🎉 模型达标！准确率: {test_acc*100:.2f}%")
else:
    print(f"\n📊 准确率: {test_acc*100:.2f}% (目标: ≥60%)")
